In [1]:
from elasticsearch import Elasticsearch

# Create an Elasticsearch client instance with authentication
es = Elasticsearch(
    "https://localhost:9200",  # Use https for SSL
    basic_auth=(
        "elastic",
        "8+6x+mXMgyoyi0F7Fi07",
    ),  # Replace with your username and password
    verify_certs=False,  # Optional: Disable SSL certificate verification for development
    ssl_show_warn=False,  # This will disable the warning
)

slop to search for phrases with words between them or swapped

In [3]:
# Adding test documents to the 'proximity' index
documents = [
    {"_id": "1", "title": "Spicy Sauce"},
    {"_id": "2", "title": "Spicy Tomato Sauce"},
    {"_id": "3", "title": "Spicy Tomato and Garlic Sauce"},
    {"_id": "4", "title": "Tomato Sauce (spicy)"},
    {"_id": "5", "title": "Spicy and very delicious Tomato Sauce"},
]

# Indexing documents
for doc in documents:
    es.index(index="proximity", id=doc["_id"], body={"title": doc["title"]})

# Refreshing the index to make the documents searchable immediately
es.indices.refresh(index="proximity")

# Running a match_phrase query with slop=1
response_slop_1 = es.search(
    index="proximity",
    body={"query": {"match_phrase": {"title": {"query": "spicy sauce", "slop": 1}}}},
)

# Running a match_phrase query with slop=2
response_slop_2 = es.search(
    index="proximity",
    body={"query": {"match_phrase": {"title": {"query": "spicy sauce", "slop": 2}}}},
)

# Displaying the results
print("Response with slop=1:", response_slop_1)
print("Response with slop=2:", response_slop_2)

Response with slop=1: {'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.11540276, 'hits': [{'_index': 'proximity', '_id': '1', '_score': 0.11540276, '_source': {'title': 'Spicy Sauce'}}, {'_index': 'proximity', '_id': '2', '_score': 0.06775388, '_source': {'title': 'Spicy Tomato Sauce'}}]}}
Response with slop=2: {'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 0.11540276, 'hits': [{'_index': 'proximity', '_id': '1', '_score': 0.11540276, '_source': {'title': 'Spicy Sauce'}}, {'_index': 'proximity', '_id': '2', '_score': 0.06775388, '_source': {'title': 'Spicy Tomato Sauce'}}, {'_index': 'proximity', '_id': '4', '_score': 0.05077119, '_source': {'title': 'Tomato Sauce (spicy)'}}]}}


affecting relevance scores, using the match phrase with slop parameter , this will boost the relevance scores for documents having the exact words in proximity

In [4]:
# Simple match query within a bool query
response_bool_match = es.search(
    index="proximity",
    body={
        "query": {"bool": {"must": [{"match": {"title": {"query": "spicy sauce"}}}]}}
    },
)

# Boosting relevance based on proximity
response_bool_match_phrase = es.search(
    index="proximity",
    body={
        "query": {
            "bool": {
                "must": [{"match": {"title": {"query": "spicy sauce"}}}],
                "should": [{"match_phrase": {"title": {"query": "spicy sauce"}}}],
            }
        }
    },
)

# Adding the slop parameter to boost relevance
response_bool_match_phrase_slop = es.search(
    index="proximity",
    body={
        "query": {
            "bool": {
                "must": [{"match": {"title": {"query": "spicy sauce"}}}],
                "should": [
                    {"match_phrase": {"title": {"query": "spicy sauce", "slop": 5}}}
                ],
            }
        }
    },
)

# Print responses
print("Response with simple match query within bool query:", response_bool_match)
print("Response with boosted relevance based on proximity:", response_bool_match_phrase)
print("Response with slop parameter in match_phrase:", response_bool_match_phrase_slop)

Response with simple match query within bool query: {'took': 22, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.11540276, 'hits': [{'_index': 'proximity', '_id': '1', '_score': 0.11540276, '_source': {'title': 'Spicy Sauce'}}, {'_index': 'proximity', '_id': '2', '_score': 0.1018082, '_source': {'title': 'Spicy Tomato Sauce'}}, {'_index': 'proximity', '_id': '4', '_score': 0.1018082, '_source': {'title': 'Tomato Sauce (spicy)'}}, {'_index': 'proximity', '_id': '3', '_score': 0.08239561, '_source': {'title': 'Spicy Tomato and Garlic Sauce'}}, {'_index': 'proximity', '_id': '5', '_score': 0.07522385, '_source': {'title': 'Spicy and very delicious Tomato Sauce'}}]}}
Response with boosted relevance based on proximity: {'took': 45, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.230

fuzzy search

In [5]:
# Searching with fuzziness set to auto (for "l0bster")
response_fuzziness_auto_l0bster = es.search(
    index="products",
    body={"query": {"match": {"name": {"query": "l0bster", "fuzziness": "auto"}}}},
)

# Searching with fuzziness set to auto (for "lobster")
response_fuzziness_auto_lobster = es.search(
    index="products",
    body={"query": {"match": {"name": {"query": "lobster", "fuzziness": "auto"}}}},
)

# Fuzziness is set per term with an integer (for "l0bster love")
response_fuzziness_int = es.search(
    index="products",
    body={
        "query": {
            "match": {
                "name": {"query": "l0bster love", "operator": "and", "fuzziness": 1}
            }
        }
    },
)

# Transpositions enabled (switching letters around, for "lvie")
response_fuzziness_transpose = es.search(
    index="products",
    body={"query": {"match": {"name": {"query": "lvie", "fuzziness": 1}}}},
)

# Disabling transpositions (for "lvie")
response_fuzziness_no_transpose = es.search(
    index="products",
    body={
        "query": {
            "match": {
                "name": {"query": "lvie", "fuzziness": 1, "fuzzy_transpositions": False}
            }
        }
    },
)

# Print each response
print("Response for fuzziness auto (l0bster):", response_fuzziness_auto_l0bster)
print("Response for fuzziness auto (lobster):", response_fuzziness_auto_lobster)
print("Response for fuzziness level 1 (l0bster love):", response_fuzziness_int)
print("Response with transpositions enabled (lvie):", response_fuzziness_transpose)
print("Response with transpositions disabled (lvie):", response_fuzziness_no_transpose)

Response for fuzziness auto (l0bster): {'took': 151, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 4.866661, 'hits': [{'_index': 'products', '_id': '19', '_score': 4.866661, '_ignored': ['description.keyword'], '_source': {'name': 'Lobster - Live', 'price': 79, 'in_stock': 43, 'sold': 370, 'tags': ['Meat', 'Seafood'], 'description': 'Integer non velit. Donec diam neque, vestibulum eget, vulputate ut, ultrices vel, augue. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Donec pharetra, magna vestibulum aliquet ultrices, erat tortor sollicitudin mi, sit amet lobortis sapien sapien non mi. Integer ac neque. Duis bibendum. Morbi non quam nec dui luctus rutrum.', 'is_active': False, 'created': '2007/08/10'}}, {'_index': 'products', '_id': '55', '_score': 4.1739345, '_source': {'name': 'Lobster - Baby Boiled', 'price': 134, 'in_stock': 41, 'sold': 2

using the fuzzy query, the query will not be analyzed which causes problems in searching. so it is better to use the match query with fuzzy parameter

In [7]:
# Fuzzy search with "LOBSTER" and fuzziness set to auto
response_fuzzy_lobster_upper = es.search(
    index="products",
    body={"query": {"fuzzy": {"name": {"value": "LOBSTER", "fuzziness": "auto"}}}},
)

# Fuzzy search with "lobster" and fuzziness set to auto
response_fuzzy_lobster_lower = es.search(
    index="products",
    body={"query": {"fuzzy": {"name": {"value": "lobster", "fuzziness": "auto"}}}},
)

# Print responses
print("Response for fuzzy search (LOBSTER):", response_fuzzy_lobster_upper)
print("Response for fuzzy search (lobster):", response_fuzzy_lobster_lower)

Response for fuzzy search (LOBSTER): {'took': 11, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
Response for fuzzy search (lobster): {'took': 18, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7, 'relation': 'eq'}, 'max_score': 5.677772, 'hits': [{'_index': 'products', '_id': '19', '_score': 5.677772, '_ignored': ['description.keyword'], '_source': {'name': 'Lobster - Live', 'price': 79, 'in_stock': 43, 'sold': 370, 'tags': ['Meat', 'Seafood'], 'description': 'Integer non velit. Donec diam neque, vestibulum eget, vulputate ut, ultrices vel, augue. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Donec pharetra, magna vestibulum aliquet ultrices, erat tortor sollicitudin mi, sit amet lobortis sapien sapien non mi. Integer ac neque. Duis bibendum. Morbi non 

while creating customer analyzer for synonyms, watch out of the order of the filters so like in this example lowercase must be before the synonymms and the synonyms matching must be all lowercase

In [8]:
# Step 1: Create an index with custom analyzer
index_name = "synonyms"
index_body = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym_test": {
                    "type": "synonym",
                    "synonyms": [
                        "awful => terrible",
                        "awesome => great, super",
                        "elasticsearch, logstash, kibana => elk",
                        "weird, strange",
                    ],
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase", "synonym_test"],
                }
            },
        }
    },
    "mappings": {
        "properties": {"description": {"type": "text", "analyzer": "my_analyzer"}}
    },
}

# Create the index
es.indices.create(index=index_name, body=index_body)


# Step 2: Test the analyzer for specific synonym words
def test_analyzer(analyzer_text):
    response = es.indices.analyze(
        index=index_name, body={"analyzer": "my_analyzer", "text": analyzer_text}
    )
    print(f"Analysis result for '{analyzer_text}':", response)


# Testing synonyms
test_analyzer("awesome")
test_analyzer("Elasticsearch")
test_analyzer("weird")
test_analyzer("Elasticsearch is awesome, but can also seem weird sometimes.")

# Step 3: Add a test document to the index
document = {
    "description": "Elasticsearch is awesome, but can also seem weird sometimes."
}
es.index(index=index_name, body=document)


# Step 4: Search for synonyms
def search_synonyms(search_text):
    response = es.search(
        index=index_name, body={"query": {"match": {"description": search_text}}}
    )
    print(f"Search results for '{search_text}':", response)


# Search for terms that should match synonyms
search_synonyms("great")
search_synonyms("awesome")

Analysis result for 'awesome': {'tokens': [{'token': 'great', 'start_offset': 0, 'end_offset': 7, 'type': 'SYNONYM', 'position': 0}, {'token': 'super', 'start_offset': 0, 'end_offset': 7, 'type': 'SYNONYM', 'position': 0}]}
Analysis result for 'Elasticsearch': {'tokens': [{'token': 'elk', 'start_offset': 0, 'end_offset': 13, 'type': 'SYNONYM', 'position': 0}]}
Analysis result for 'weird': {'tokens': [{'token': 'weird', 'start_offset': 0, 'end_offset': 5, 'type': '<ALPHANUM>', 'position': 0}, {'token': 'strange', 'start_offset': 0, 'end_offset': 5, 'type': 'SYNONYM', 'position': 0}]}
Analysis result for 'Elasticsearch is awesome, but can also seem weird sometimes.': {'tokens': [{'token': 'elk', 'start_offset': 0, 'end_offset': 13, 'type': 'SYNONYM', 'position': 0}, {'token': 'is', 'start_offset': 14, 'end_offset': 16, 'type': '<ALPHANUM>', 'position': 1}, {'token': 'great', 'start_offset': 17, 'end_offset': 24, 'type': 'SYNONYM', 'position': 2}, {'token': 'super', 'start_offset': 17, 'e

adding synonyms from txt file

In [13]:
from elasticsearch.exceptions import NotFoundError

# Define the index name
index_name = "synonyms"

# Delete the existing index if it exists
try:
    es.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")
except NotFoundError:
    print(f"Index {index_name} does not exist, nothing to delete.")

# Define index with custom analyzer referencing the synonyms file
index_body = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym_test": {
                    "type": "synonym",
                    "synonyms_path": "./synonyms.txt",  # Path within Elasticsearch config directory
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase", "synonym_test"],
                }
            },
        }
    },
    "mappings": {
        "properties": {"description": {"type": "text", "analyzer": "my_analyzer"}}
    },
}

# Create the index with the custom analyzer
es.indices.create(index=index_name, body=index_body)
print(f"Created index: {index_name}")


# Function to test the custom analyzer
def test_analyzer(analyzer_text):
    response = es.indices.analyze(
        index=index_name, body={"analyzer": "my_analyzer", "text": analyzer_text}
    )
    print(f"Analysis result for '{analyzer_text}':", response)


# Testing the analyzer for synonym expansion with example text
test_analyzer("Elasticsearch")

Index synonyms does not exist, nothing to delete.
Created index: synonyms
Analysis result for 'Elasticsearch': {'tokens': [{'token': 'elk', 'start_offset': 0, 'end_offset': 13, 'type': 'SYNONYM', 'position': 0}]}


In [17]:
# Define index name
index_name = "highlighting"

# Create index if it doesn't exist
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)
    print(f"Created index: {index_name}")

# Adding a test document
doc_id = 1
test_document = {
    "description": "Let me tell you a story about Elasticsearch. It's a full-text search engine that is built on Apache Lucene. It's really easy to use, but also packs lots of advanced features that you can use to tweak its searching capabilities. Lots of well-known and established companies use Elasticsearch, and so should you!"
}

# Indexing the document
es.index(index=index_name, id=doc_id, body=test_document)
print(f"Added document with ID {doc_id}.")

# Highlighting matches within the description field
search_query = {
    "_source": False,
    "query": {"match": {"description": "Elasticsearch story"}},
    "highlight": {"fields": {"description": {}}},
}

highlight_response = es.search(index=index_name, body=search_query)
print("Highlighting matches:")
print(highlight_response)

# Specifying a custom tag
custom_highlight_query = {
    "_source": False,
    "query": {"match": {"description": "Elasticsearch story"}},
    "highlight": {
        "pre_tags": ["<strong>"],
        "post_tags": ["</strong>"],
        "fields": {"description": {}},
    },
}

custom_highlight_response = es.search(index=index_name, body=custom_highlight_query)
print("Highlighting matches with custom tags:")
print(custom_highlight_response)

Added document with ID 1.
Highlighting matches:
{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.68324494, 'hits': [{'_index': 'highlighting', '_id': '1', '_score': 0.68324494, '_ignored': ['description.keyword'], 'highlight': {'description': ['Let me tell you a <em>story</em> about <em>Elasticsearch</em>.', 'Lots of well-known and established companies use <em>Elasticsearch</em>, and so should you!']}}]}}
Highlighting matches with custom tags:
{'took': 15, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.68324494, 'hits': [{'_index': 'highlighting', '_id': '1', '_score': 0.68324494, '_ignored': ['description.keyword'], 'highlight': {'description': ['Let me tell you a <strong>story</strong> about <strong>Elasticsearch</strong>.', 'Lots of well-known and established c

add stemmer also

In [19]:
# Define index name
index_name = "stemming_test"

# Create index with custom analyzer
if not es.indices.exists(index=index_name):
    index_body = {
        "settings": {
            "analysis": {
                "filter": {
                    "synonym_test": {
                        "type": "synonym",
                        "synonyms": ["firm => company", "love, enjoy"],
                    },
                    "stemmer_test": {"type": "stemmer", "name": "english"},
                },
                "analyzer": {
                    "my_analyzer": {
                        "tokenizer": "standard",
                        "filter": ["lowercase", "synonym_test", "stemmer_test"],
                    }
                },
            }
        },
        "mappings": {
            "properties": {"description": {"type": "text", "analyzer": "my_analyzer"}}
        },
    }

    es.indices.create(index=index_name, body=index_body)
    print(f"Created index: {index_name}")

# Adding a test document
doc_id = 1
test_document = {"description": "I love working for my firm!"}

# Indexing the document
es.index(index=index_name, id=doc_id, body=test_document)
print(f"Added document with ID {doc_id}.")

# Matching the document with the base word (work)
search_query_1 = {"query": {"match": {"description": "enjoy work"}}}

search_response_1 = es.search(index=index_name, body=search_query_1)
print("Search results for 'enjoy work':")
print(search_response_1)

# The query is stemmed, so the document still matches
search_query_2 = {"query": {"match": {"description": "love working"}}}

search_response_2 = es.search(index=index_name, body=search_query_2)
print("Search results for 'love working':")
print(search_response_2)

# Synonyms and stemmed words are still highlighted
highlight_query = {
    "query": {"match": {"description": "enjoy work"}},
    "highlight": {"fields": {"description": {}}},
}

highlight_response = es.search(index=index_name, body=highlight_query)
print("Highlighting matches for 'enjoy work':")
print(highlight_response)

Added document with ID 1.
Search results for 'enjoy work':
{'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.7176595, 'hits': [{'_index': 'stemming_test', '_id': '1', '_score': 0.7176595, '_source': {'description': 'I love working for my firm!'}}]}}
Search results for 'love working':
{'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.7176595, 'hits': [{'_index': 'stemming_test', '_id': '1', '_score': 0.7176595, '_source': {'description': 'I love working for my firm!'}}]}}
Highlighting matches for 'enjoy work':
{'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.7176595, 'hits': [{'_index': 'stemming_test', '_id': '1', '_score': 0.7176595, '_sour